# 📘 Table des Matières — Notebook 03

1. [Imports et Configuration](#Cellule-1---Imports-et-Configuration)
2. [Fonctions Utilitaires](#Cellule-2---Fonctions-Utilitaires)
3. [Chargement du Modèle V2 (GAN + CBAM)](#Cellule-3---Chargement-du-Modèle-V2-GAN--CBAM)
4. [Génération d'Échantillons V2](#Cellule-4---Génération-d'Échantillons-V2)
5. [Chargement du Modèle V3 (DDPM)](#Cellule-5---Chargement-du-Modèle-V3-DDPM)
6. [Génération d'Échantillons V3 (Diffusion)](#Cellule-6---Génération-d'Échantillons-V3-Diffusion)
7. [Comparaison Visuelle Directe](#Cellule-7---Comparaison-Visuelle-Directe)
8. [Analyse Statistique des Générations](#Cellule-8---Analyse-Statistique-des-Générations)
9. [Analyse de Diversité des Générations](#Cellule-9---Analyse-de-Diversité-des-Générations)
10. [Conclusions et Observations](#Cellule-10---Conclusions-et-Observations)


# Notebook 03 - GAN V2 avec Attention et Modèle de Diffusion

Dans ce notebook, j'analyse et compare les performances de mes modèles V2 (GAN + CBAM) et V3 (DDPM).

**Objectifs :**
- Charger les modèles entraînés V2 et V3
- Générer des échantillons avec chaque architecture
- Visualiser les différences qualitatives
- Comparer les métriques de performance


## Cellule 1 - Imports et Configuration


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

# J'ajoute le chemin racine pour importer mes modules
sys.path.append(str(Path.cwd().parent))

from models.generator_attention import build_generator_attention
from models.diffusion import build_unet_conditional, generate_samples
from utils.attributes import sample_random_attributes
from utils.config import *

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")


## Cellule 2 - Fonctions Utilitaires


In [ ]:
def denormalize_images(images):
    """
    Je dénormalise les images de [-1, 1] vers [0, 1] pour affichage.
    """
    return (images + 1.0) / 2.0

def plot_image_grid(images, title, rows=4, cols=4, figsize=(12, 12)):
    """
    J'affiche une grille d'images avec un titre personnalisé.
    """
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten()
    
    images_denorm = denormalize_images(images[:rows*cols])
    
    for i, (ax, img) in enumerate(zip(axes, images_denorm)):
        ax.imshow(np.clip(img, 0, 1))
        ax.axis('off')
        ax.set_title(f'Sample {i+1}', fontsize=8)
    
    plt.suptitle(title, fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()

def compare_models_side_by_side(v2_images, v3_images, num_pairs=4):
    """
    Je compare côte à côte des échantillons V2 et V3.
    """
    fig, axes = plt.subplots(num_pairs, 2, figsize=(8, num_pairs*2))
    
    v2_denorm = denormalize_images(v2_images[:num_pairs])
    v3_denorm = denormalize_images(v3_images[:num_pairs])
    
    for i in range(num_pairs):
        axes[i, 0].imshow(np.clip(v2_denorm[i], 0, 1))
        axes[i, 0].set_title('V2 (GAN + CBAM)', fontweight='bold')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(np.clip(v3_denorm[i], 0, 1))
        axes[i, 1].set_title('V3 (DDPM)', fontweight='bold')
        axes[i, 1].axis('off')
    
    plt.suptitle('Comparaison Directe V2 vs V3', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("Fonctions utilitaires chargées avec succès")


## Cellule 3 - Chargement du Modèle V2 (GAN + CBAM)


In [ ]:
# Je charge le générateur V2 avec attention
v2_checkpoint_path = '../outputs/checkpoints_v2/generator_final.h5'

if os.path.exists(v2_checkpoint_path):
    print("Chargement du générateur V2...")
    generator_v2 = build_generator_attention()
    generator_v2.load_weights(v2_checkpoint_path)
    print(f"Modèle V2 chargé: {generator_v2.count_params():,} paramètres")
    generator_v2.summary()
else:
    print(f"ERREUR: Checkpoint V2 introuvable à {v2_checkpoint_path}")
    print("Je dois d'abord entraîner le modèle V2 avec training/train_gan_attention.py")


## Cellule 4 - Génération d'Échantillons V2


In [ ]:
# Je génère 16 images avec V2
num_samples = 16
noise_v2 = np.random.randn(num_samples, NOISE_DIM).astype('float32')

print("Génération en cours avec V2 (GAN + CBAM)...")
v2_generated = generator_v2.predict(noise_v2, verbose=0)
print(f"Shape des images générées: {v2_generated.shape}")
print(f"Plage de valeurs: [{v2_generated.min():.3f}, {v2_generated.max():.3f}]")

# J'affiche la grille
plot_image_grid(v2_generated, 'Génération V2 - GAN avec Attention CBAM', rows=4, cols=4)


## Cellule 5 - Chargement du Modèle V3 (DDPM)


In [ ]:
# Je charge le U-Net conditionnel V3
v3_checkpoint_path = '../outputs/checkpoints_v3/unet_final.h5'

if os.path.exists(v3_checkpoint_path):
    print("Chargement du U-Net V3...")
    unet_v3 = build_unet_conditional()
    unet_v3.load_weights(v3_checkpoint_path)
    print(f"Modèle V3 chargé: {unet_v3.count_params():,} paramètres")
    unet_v3.summary()
else:
    print(f"ERREUR: Checkpoint V3 introuvable à {v3_checkpoint_path}")
    print("Je dois d'abord entraîner le modèle V3 avec training/train_ddpm.py")


## Cellule 6 - Génération d'Échantillons V3 (Diffusion)

**Note :** la génération avec DDPM est plus lente car elle nécessite 1000 étapes de débruitage itératif.


In [ ]:
# Je génère des attributs aléatoires pour le conditionnement
v3_attributes = sample_random_attributes(num_samples, seed=42)
v3_attributes_tensor = tf.constant(v3_attributes, dtype=tf.float32)

print("Génération en cours avec V3 (DDPM - cela peut prendre 1-2 minutes)...")
import time
start_time = time.time()

v3_generated = generate_samples(unet_v3, num_samples, v3_attributes_tensor)

elapsed = time.time() - start_time
print(f"Génération terminée en {elapsed:.2f}s ({elapsed/num_samples:.2f}s par image)")
print(f"Shape des images générées: {v3_generated.shape}")
print(f"Plage de valeurs: [{v3_generated.numpy().min():.3f}, {v3_generated.numpy().max():.3f}]")

# J'affiche la grille
plot_image_grid(v3_generated.numpy(), 'Génération V3 - Modèle de Diffusion DDPM', rows=4, cols=4)


## Cellule 7 - Comparaison Visuelle Directe


In [ ]:
# Je compare 4 paires d'images côte à côte
compare_models_side_by_side(v2_generated, v3_generated.numpy(), num_pairs=4)


## Cellule 8 - Analyse Statistique des Générations


In [ ]:
# J'analyse les statistiques des pixels générés
v2_stats = {
    'mean': np.mean(v2_generated),
    'std': np.std(v2_generated),
    'min': np.min(v2_generated),
    'max': np.max(v2_generated)
}

v3_stats = {
    'mean': np.mean(v3_generated.numpy()),
    'std': np.std(v3_generated.numpy()),
    'min': np.min(v3_generated.numpy()),
    'max': np.max(v3_generated.numpy())
}

# J'affiche les statistiques
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme V2
axes[0].hist(v2_generated.flatten(), bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0].set_title('Distribution des Pixels V2 (GAN + CBAM)', fontweight='bold')
axes[0].set_xlabel('Valeur du pixel')
axes[0].set_ylabel('Fréquence')
axes[0].axvline(v2_stats['mean'], color='red', linestyle='--', linewidth=2, label=f"Moyenne: {v2_stats['mean']:.3f}")
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Histogramme V3
axes[1].hist(v3_generated.numpy().flatten(), bins=50, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Distribution des Pixels V3 (DDPM)', fontweight='bold')
axes[1].set_xlabel('Valeur du pixel')
axes[1].set_ylabel('Fréquence')
axes[1].axvline(v3_stats['mean'], color='red', linestyle='--', linewidth=2, label=f"Moyenne: {v3_stats['mean']:.3f}")
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# J'affiche le tableau comparatif
print("\n" + "="*60)
print("COMPARAISON STATISTIQUE V2 vs V3")
print("="*60)
print(f"{'Métrique':<15} {'V2 (GAN+CBAM)':<20} {'V3 (DDPM)':<20}")
print("-"*60)
print(f"{'Moyenne':<15} {v2_stats['mean']:<20.4f} {v3_stats['mean']:<20.4f}")
print(f"{'Écart-type':<15} {v2_stats['std']:<20.4f} {v3_stats['std']:<20.4f}")
print(f"{'Minimum':<15} {v2_stats['min']:<20.4f} {v3_stats['min']:<20.4f}")
print(f"{'Maximum':<15} {v2_stats['max']:<20.4f} {v3_stats['max']:<20.4f}")
print("="*60)


## Cellule 9 - Analyse de Diversité des Générations


In [ ]:
# Je calcule la diversité en mesurant les différences entre échantillons
def calculate_diversity(images):
    """
    Je calcule la diversité comme l'écart-type moyen entre toutes les paires d'images.
    Plus le score est élevé, plus les images sont différentes les unes des autres.
    """
    n = images.shape[0]
    differences = []
    
    for i in range(n):
        for j in range(i+1, n):
            diff = np.mean(np.abs(images[i] - images[j]))
            differences.append(diff)
    
    return np.mean(differences), np.std(differences)

v2_diversity_mean, v2_diversity_std = calculate_diversity(v2_generated)
v3_diversity_mean, v3_diversity_std = calculate_diversity(v3_generated.numpy())

# Je visualise la comparaison
fig, ax = plt.subplots(figsize=(10, 6))

models = ['V2 (GAN+CBAM)', 'V3 (DDPM)']
diversity_means = [v2_diversity_mean, v3_diversity_mean]
diversity_stds = [v2_diversity_std, v3_diversity_std]

x_pos = np.arange(len(models))
colors = ['#1f77b4', '#2ca02c']

bars = ax.bar(x_pos, diversity_means, yerr=diversity_stds, 
              capsize=10, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

# J'ajoute les valeurs sur les barres
for i, (bar, mean, std) in enumerate(zip(bars, diversity_means, diversity_stds)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + std,
            f'{mean:.4f}\n±{std:.4f}',
            ha='center', va='bottom', fontweight='bold', fontsize=10)

ax.set_ylabel('Score de Diversité (Distance Moyenne)', fontsize=12, fontweight='bold')
ax.set_title('Comparaison de la Diversité des Générations\n(Plus haut = Plus de variabilité)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x_pos)
ax.set_xticklabels(models, fontsize=11, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)

plt.tight_layout()
plt.show()

print(f"\nScore de diversité V2: {v2_diversity_mean:.4f} ± {v2_diversity_std:.4f}")
print(f"Score de diversité V3: {v3_diversity_mean:.4f} ± {v3_diversity_std:.4f}")

if v3_diversity_mean > v2_diversity_mean:
    print("\nRésultat: V3 génère des visages plus diversifiés que V2")
else:
    print("\nRésultat: V2 génère des visages plus diversifiés que V3")


## Cellule 10 - Conclusions et Observations


In [ ]:
# Je sauvegarde mes résultats pour le rapport final
results = {
    'v2_stats': v2_stats,
    'v3_stats': v3_stats,
    'v2_diversity': {'mean': float(v2_diversity_mean), 'std': float(v2_diversity_std)},
    'v3_diversity': {'mean': float(v3_diversity_mean), 'std': float(v3_diversity_std)}
}

import json
output_path = '../outputs/comparison_v2_v3.json'
with open(output_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nRésultats sauvegardés dans {output_path}")
print("\nNotebook 03 terminé avec succès !")
